In [36]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [37]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [38]:
len(words)

32033

In [39]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [40]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
	X, Y = [], []

	for w in words:
		context = [0] * block_size
		for ch in w + '.':
			ix = stoi[ch]
			X.append(context)
			Y.append(ix)
			context = context[1:] + [ix] # crop and append

	X = torch.tensor(X)
	Y = torch.tensor(Y)
	print(X.shape, Y.shape)
	return X, Y

import random

random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1]) # 80%
Xdev, Ydev = build_dataset(words[n1:n2]) # 10%
Xte, Yte = build_dataset(words[n2:]) # 10%

torch.Size([182510, 3]) torch.Size([182510])
torch.Size([22736, 3]) torch.Size([22736])
torch.Size([22900, 3]) torch.Size([22900])


In [41]:
# MLP
n_embd = 10
n_hidden = 200

C = torch.randn((vocab_size, n_embd))
W1 = torch.randn((n_embd * block_size, n_hidden)) * (5/3)/((n_embd * block_size)**0.5)
# b1 = torch.randn(n_hidden) * 0.01
W2 = torch.randn((n_hidden, vocab_size)) * 0.01
b2 = torch.randn(vocab_size) * 0

# BatchNorm parameters
bngain = torch.ones((1, n_hidden))
bnbias = torch.zeros((1, n_hidden))
bnmean_running = torch.zeros((1, n_hidden))
bnstd_running = torch.ones((1, n_hidden))

parameters = [C, W1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
	p.requires_grad = True

12097


In [50]:
# same optimization as last time
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):

	# minibatch construct
	ix = torch.randint(0, Xtr.shape[0], (batch_size,))
	Xb, Yb = Xtr[ix], Ytr[ix] # batch X, Y

	# forward pass
	emb = C[Xb] # embed the characters into vectors
	embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
	# Linear layer
	hpreact = embcat @ W1 #+ b1 # hidden layer pre-activation
	# BatchNorm layer
	# -----------------------------------------------------------
	bnmeani = hpreact.mean(0, keepdim=True)
	bnstdi = hpreact.std(0, keepdim=True)
	hpreact = bngain * (hpreact - bnmeani) / bnstdi + bnbias

	with torch.no_grad():
		bnmean_running = 0.999 * bnmean_running + 0.001 * bnmeani
		bnstd_running = 0.999 * bnstd_running + 0.001 * bnstdi

	# -----------------------------------------------------------
	# Non-linearity
	h = torch.tanh(hpreact) # hidden layer
	logits = h @ W2 + b2 # output layer
	loss = F.cross_entropy(logits, Yb) # loss function

	# backward pass
	for p in parameters:
		p.grad = None
	loss.backward()

	# update 
	lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
	for p in parameters:
		p.data += -lr * p.grad

	# track status
	if i % 10000 == 0: # print every once in a while
		print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
	lossi.append(loss.log10().item())

	# break


      0/ 200000: 2.1136
  10000/ 200000: 2.2655
  20000/ 200000: 1.9276
  30000/ 200000: 2.2278
  40000/ 200000: 1.8934
  50000/ 200000: 2.5127
  60000/ 200000: 2.1503
  70000/ 200000: 1.9038
  80000/ 200000: 2.2180
  90000/ 200000: 2.2857
 100000/ 200000: 2.1685
 110000/ 200000: 1.9362
 120000/ 200000: 2.0862
 130000/ 200000: 2.3718
 140000/ 200000: 2.0454
 150000/ 200000: 2.1152
 160000/ 200000: 1.9165
 170000/ 200000: 2.3683
 180000/ 200000: 1.8596
 190000/ 200000: 1.9260


In [47]:
# calibrate the batch norm at the end of training

with torch.no_grad():
	# pass the training set through
	emb = C[Xtr]
	embcat = emb.view(emb.shape[0], -1)
	hpreact = embcat @ W1
	# measure the mean/std over the entire training set
	bnmean = hpreact.mean(0, keepdim=True)
	bnstd = hpreact.std(0, keepdim=True)

In [48]:
@torch.no_grad()
def split_loss(split):
	x, y = {
		'train': (Xtr, Ytr),
		'val': (Xdev, Ydev),
		'test': (Xte, Yte)
	}[split]
	emb = C[x] # (N, block_size, n_embd)
	embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
	hpreact = embcat @ W1
	# hpreact = bngain * (hpreact - hpreact.mean(0, keepdim=True)) / hpreact.std(0, keepdim=True) + bnbias
	hpreact = bngain * (hpreact - bnmean_running) / bnstd_running + bnbias
	h = torch.tanh(embcat @ W1) # (N, n_hidden)
	logits = h @ W2 + b2 # (N, vocab_size)
	loss = F.cross_entropy(logits, y)
	print(split, loss.item())

split_loss('train')
split_loss('val')

train 3.4050610065460205
val 3.4638779163360596


In [49]:
# sample from the model
for _ in range(20):
    
	out = []
	context = [0] * block_size

	while True:
		# forward pass the neural net
		emb = C[torch.tensor([context])] # (1, block_size, n_embd)
		h = torch.tanh(emb.view(1, -1) @ W1)
		logits = h @ W2 + b2
		probs = F.softmax(logits, dim=1)
		# sample from the distribution
		ix = torch.multinomial(probs, num_samples=1).item()
		# shift the context window and track the samples
		context = context[1:] + [ix]
		out.append(ix)
		# if we sample the special '.' token, break
		if ix == 0:
			break
	
	print(''.join(itos[i] for i in out))

kastamodrrirushivagiv.
zavedgkenslinalydnsldtamlystemmetzig.
xuctorionovelsonzam.
malimthtyanshlylisdrushmwanverlysxkiexlulwawwendamir.
xuctonlaningshasvyn.
khylannejahmzda.
jshlmaw.
hvilystonwanzlillawynvigaston.
qustipustifarval.
hvel.
sudricusmanlystwanilynalifajhestifgsniarrushwsanwaystoncevvidlapgayllixshlliszavionselyzxandrahmrroslptod.
kyvionlylynishaddiwhodnceldassamvyoningnathyldsleyamirlinaintquenamsterislynannamrolantworamsydnslxunasmah.
varemijlaw.
huphtos.
supuldestaffynstonwelannagricustifrorwinslinevlgyngiyndonxastifaygol.
kaleimallyvia.
varislynamraflvid.
gsy.
cayndlysterdynlamah.
qun.
